In [1]:
import numpy as np
import torch
import torch_geometric.datasets as datasets
import torch_geometric.data as data
import torch_geometric.transforms as transforms
import networkx as nx
from torch_geometric.utils.convert import to_networkx

# Dataset

In [2]:
import pandas as pd
node=pd.read_csv('user_nodes_data.csv')

In [3]:
node.head(2)

,UserId,CurrentProficiency,PreviousProficiency,CurrentAccuracy,PreviousAccuracy,CoveragePercentage,AccuracyPercentage,TotalQuestions,CorrectQuestions,MedianPercentage,PreviousAchievement,IsSkillStuck,SkillId
0,2881,1,1.0,36.36364,9.090909,100.0,36.36364,11.0,4.0,100.0,NaN,NaN,433
1,2903,4,4.0,100.00000,100.000000,100.0,100.00000,10.0,10.0,100.0,NaN,NaN,432


In [4]:
y=node.SkillId.tolist()
# print(y)


In [5]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)

LabelEncoder()

In [6]:
# le.classes_

In [7]:
y_en=le.transform(y)

In [8]:
y_en

array([ 2,  1,  1, ..., 16, 16, 23], dtype=int64)

In [9]:
# print(node.SkillId.unique())

In [10]:
node=node.drop(['UserId','SkillId'],axis=1)

In [11]:
node=node.fillna(0)

In [12]:
node.head(1)

,CurrentProficiency,PreviousProficiency,CurrentAccuracy,PreviousAccuracy,CoveragePercentage,AccuracyPercentage,TotalQuestions,CorrectQuestions,MedianPercentage,PreviousAchievement,IsSkillStuck
0,1,1.0,36.36364,9.090909,100.0,36.36364,11.0,4.0,100.0,0.0,0.0


In [13]:
features = node.values.tolist()
# features

In [14]:
features

[[1.0, 1.0, 36.36364, 9.090909, 100.0, 36.36364, 11.0, 4.0, 100.0, 0.0, 0.0],
 [4.0, 4.0, 100.0, 100.0, 100.0, 100.0, 10.0, 10.0, 100.0, 0.0, 0.0],
 [3.0, 0.0, 100.0, 0.0, 60.0, 100.0, 6.0, 6.0, 100.0, 0.0, 0.0],
 [2.0, 0.0, 100.0, 0.0, 54.54546, 100.0, 6.0, 6.0, 100.0, 0.0, 0.0],
 [4.0, 4.0, 100.0, 100.0, 100.0, 100.0, 11.0, 11.0, 100.0, 0.0, 0.0],
 [4.0, 2.0, 100.0, 66.66666, 100.0, 100.0, 10.0, 10.0, 100.0, 0.0, 0.0],
 [4.0, 2.0, 81.81818, 83.33334, 100.0, 81.81818, 11.0, 9.0, 100.0, 0.0, 0.0],
 [4.0, 4.0, 100.0, 100.0, 100.0, 100.0, 10.0, 10.0, 100.0, 0.0, 0.0],
 [2.0, 2.0, 60.0, 60.0, 100.0, 60.0, 10.0, 6.0, 100.0, 0.0, 0.0],
 [4.0, 2.0, 100.0, 100.0, 100.0, 100.0, 14.0, 14.0, 100.0, 0.0, 0.0],
 [4.0, 0.0, 100.0, 0.0, 100.0, 100.0, 10.0, 10.0, 100.0, 0.0, 0.0],
 [4.0, 4.0, 100.0, 100.0, 100.0, 100.0, 10.0, 10.0, 100.0, 0.0, 0.0],
 [4.0, 4.0, 100.0, 100.0, 100.0, 100.0, 11.0, 11.0, 100.0, 0.0, 0.0],
 [4.0, 4.0, 100.0, 100.0, 100.0, 100.0, 10.0, 10.0, 100.0, 0.0, 0.0],
 [4.0, 1.0, 1

In [15]:
# gDf

In [16]:
ed=pd.read_csv('final_user_edges.csv')
ed.head(1)

,Unnamed: 0.1,Unnamed: 0,skill_id,preq
0,0,0,2881,2903


In [17]:
sk_i=ed.skill_id.tolist()
preq=ed.preq.tolist()

In [18]:
sk_i

[2881,
 2903,
 2939,
 2971,
 2999,
 2961,
 3090,
 2884,
 2942,
 3189,
 3188,
 3193,
 3191,
 3221,
 3137,
 3216,
 3223,
 3222,
 3240,
 3242,
 3250,
 3175,
 3255,
 3265,
 3245,
 3270,
 3272,
 3252,
 1111,
 3267,
 3286,
 3298,
 3293,
 3302,
 3303,
 3306,
 2915,
 3310,
 3239,
 3256,
 3186,
 2892,
 3342,
 3330,
 3345,
 3337,
 2632,
 3217,
 3253,
 3084,
 3289,
 3030,
 3307,
 3312,
 1823,
 3341,
 3370,
 2932,
 3424,
 3350,
 3450,
 3475,
 3476,
 1846,
 3489,
 3273,
 1545,
 3451,
 3527,
 3460,
 3439,
 3538,
 3387,
 3514,
 3557,
 3585,
 3609,
 3622,
 3608,
 3624,
 3619,
 3657,
 3438,
 3309,
 3720,
 3715,
 3688,
 3666,
 3018,
 3769,
 3777,
 3775,
 3710,
 3588,
 3845,
 3848,
 3561,
 3677,
 3758,
 3862,
 3431,
 3868,
 3863,
 3805,
 3528,
 3921,
 3917,
 3739,
 3923,
 3928,
 3900,
 3935,
 3734,
 3368,
 4034,
 4035,
 3548,
 4045,
 4052,
 4062,
 3525,
 3902,
 4077,
 3843,
 4104,
 4120,
 3761,
 2878,
 3283,
 3986,
 4154,
 3995,
 4178,
 4192,
 4223,
 4217,
 4225,
 3316,
 4097,
 4218,
 4019,
 4245,
 4253,

In [19]:
# preq

In [20]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
sk_i=le.fit_transform(sk_i)


In [21]:
preq=le.fit_transform(preq)

In [22]:
print(max(sk_i))
print(min(sk_i))

11233
0


In [23]:
len(sk_i)

189113

In [24]:
max(preq)

10109

In [25]:
len(preq)

189113

In [26]:
features

[[1.0, 1.0, 36.36364, 9.090909, 100.0, 36.36364, 11.0, 4.0, 100.0, 0.0, 0.0],
 [4.0, 4.0, 100.0, 100.0, 100.0, 100.0, 10.0, 10.0, 100.0, 0.0, 0.0],
 [3.0, 0.0, 100.0, 0.0, 60.0, 100.0, 6.0, 6.0, 100.0, 0.0, 0.0],
 [2.0, 0.0, 100.0, 0.0, 54.54546, 100.0, 6.0, 6.0, 100.0, 0.0, 0.0],
 [4.0, 4.0, 100.0, 100.0, 100.0, 100.0, 11.0, 11.0, 100.0, 0.0, 0.0],
 [4.0, 2.0, 100.0, 66.66666, 100.0, 100.0, 10.0, 10.0, 100.0, 0.0, 0.0],
 [4.0, 2.0, 81.81818, 83.33334, 100.0, 81.81818, 11.0, 9.0, 100.0, 0.0, 0.0],
 [4.0, 4.0, 100.0, 100.0, 100.0, 100.0, 10.0, 10.0, 100.0, 0.0, 0.0],
 [2.0, 2.0, 60.0, 60.0, 100.0, 60.0, 10.0, 6.0, 100.0, 0.0, 0.0],
 [4.0, 2.0, 100.0, 100.0, 100.0, 100.0, 14.0, 14.0, 100.0, 0.0, 0.0],
 [4.0, 0.0, 100.0, 0.0, 100.0, 100.0, 10.0, 10.0, 100.0, 0.0, 0.0],
 [4.0, 4.0, 100.0, 100.0, 100.0, 100.0, 10.0, 10.0, 100.0, 0.0, 0.0],
 [4.0, 4.0, 100.0, 100.0, 100.0, 100.0, 11.0, 11.0, 100.0, 0.0, 0.0],
 [4.0, 4.0, 100.0, 100.0, 100.0, 100.0, 10.0, 10.0, 100.0, 0.0, 0.0],
 [4.0, 1.0, 1

In [27]:
# x_test= torch.tensor([[1,2],[1,2],[1,2]], dtype=torch.float)

In [28]:
x = torch.tensor(features, dtype=torch.float)

In [29]:
y_en = torch.tensor(y_en, dtype=torch.float)

In [30]:
y_en

tensor([ 2.,  1.,  1.,  ..., 16., 16., 23.])

In [31]:
import torch
from torch_geometric.data import Data
edge_index = torch.tensor([preq,sk_i], dtype=torch.long)
dt=Data(x=x, y=y_en, edge_index=edge_index)

C:\Users\tubaid\AppData\Local\Temp\ipykernel_6052\634187113.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:210.)
  edge_index = torch.tensor([preq,sk_i], dtype=torch.long)


In [32]:
train_mask=[]
for i in range(120708):
    if i<=84495:
        train_mask.append(True)
    else:
        train_mask.append(False)

In [33]:
train_mask=torch.tensor(train_mask)

In [34]:
train_mask

tensor([ True,  True,  True,  ..., False, False, False])

In [35]:
val_mask=[]
# 10870
for i in range(120708):
    if (i<=108637 and i>=84495):
        val_mask.append(True)
    else:
        val_mask.append(False)

In [36]:
val_mask=torch.tensor(val_mask)

In [37]:
val_mask

tensor([False, False, False,  ..., False, False, False])

In [38]:
test_mask=[]
for i in range(120708):
    if i<=108637:
        test_mask.append(False)
    else:
        test_mask.append(True)


In [39]:
test_mask=torch.tensor(test_mask)

In [40]:
test_mask

tensor([False, False, False,  ...,  True,  True,  True])

In [41]:
# sk_i=le.transform(sk_i)

In [42]:
# train_mask=torch.tensor(np.random.choice([True, False], size=(120708,),p=[0.5,0.5]))
# val_mask=torch.tensor(np.random.choice([True, False], size=(120708,),p=[0.5,0.5]))
# test_mask=torch.tensor(np.random.choice([True, False], size=(120708),p=[0.2,0.8]))

In [43]:
# train[]=0
# train.append('true')

In [44]:
# train_mask = torch.tensor(train_mask)

In [45]:
y_en = y_en.type(torch.LongTensor)

In [46]:
dt=Data(x=x, y=y_en, edge_index=edge_index,
train_mask=train_mask,val_mask=val_mask,test_mask=test_mask)
dt

Data(x=[120708, 11], edge_index=[2, 189113], y=[120708], train_mask=[120708], val_mask=[120708], test_mask=[120708])

In [47]:
dt.y

tensor([ 2,  1,  1,  ..., 16, 16, 23])

# Model

In [48]:
from torch_geometric.datasets import Planetoid
import torch
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

In [49]:
class GCNConv(MessagePassing):
    # Applies a linear tranformation to the incoming data
    def __init__(self, in_channels, out_channels):
        super(GCNConv, self).__init__(aggr='add')  # "Add" aggregation
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def forward(self, x, edge_index):
        # Step 1: Add self-loops
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        # Step 2: Multiply with weights
        x = self.lin(x)

        # Step 3: Calculate the normalization
        row, col = edge_index
        deg = degree(row, x.size(0), dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        # Step 4: Propagate the embeddings to the next layer
        return self.propagate(edge_index, size=(x.size(0), x.size(0)), x=x,
                              norm=norm)

    def message(self, x_j, norm):
        # Normalize node features.
        return norm.view(-1, 1) * x_j


In [50]:
class Net(torch.nn.Module):

    def __init__(self, dataset):
        super(Net, self).__init__()
        # two convolution layers of GCN
        # first represnt the input and the second represnt the output layer
        # dataset.num_node_features== number of input
        # 16 hidden layers
        # dataset.num_classes==outputclass
        
        self.conv1 = GCNConv(11, 16)
        self.conv2 = GCNConv(16, 163)
#         self.conv1 = GCNConv(11, 256)
#         self.conv2 = GCNConv(256, 512)
#         self.conv3 = GCNConv(512, 512)
#         self.conv4 = GCNConv(512, 1024)
#         self.conv5 = GCNConv(1024, 163)
    # represent the weights updates
    # weights in output
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
#         print ('all values of  x:',x)
#         print ('all edges:',edge_index)
        
        x = self.conv1(x, edge_index)
#         print ('results after the  1st convn:',x)
#         print("First of x:",data.x[1])
        x = F.leaky_relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        
#         x = F.leaky_relu(x)
#         x = F.dropout(x, training=self.training)
#         x = self.conv3(x, edge_index)
        
#         x = F.leaky_relu(x)
# #         x = F.dropout(x, training=self.training)
#         x = self.conv4(x, edge_index)
        
#         x = F.leaky_relu(x)
# #         x = F.dropout(x, training=self.training)
#         x = self.conv5(x, edge_index)
        
        
#         print('softmax_output:',F.log_softmax.x)
        return F.log_softmax(x, dim=1)
#         return F.CrossEntropyLoss(x, dim=1)
        
# torch.nn.CrossEntropyLoss



In [51]:
def plot_dataset(dataset):
    edges_raw = dataset.data.edge_index.numpy()
    edges = [(x, y) for x, y in zip(edges_raw[0, :], edges_raw[1, :])]
    labels = dataset.data.y.numpy()

    G = nx.Graph()
    G.add_nodes_from(list(range(np.max(edges_raw))))
    G.add_edges_from(edges)
    plt.subplot(111)
    options = {
                'node_size': 30,
                'width': 0.2,
    }
    nx.draw(G, with_labels=False, node_color=labels.tolist(), cmap=plt.cm.tab10, font_weight='bold', **options)
    plt.show()


In [52]:
def test(data, train=True):
    model.eval()

    correct = 0
    pred = model(data).max(dim=1)[1]

    if train:
        correct += pred[data.train_mask].eq(data.y[data.train_mask]).sum().item()
        return correct / (len(data.y[data.train_mask]))
    else:
        correct += pred[data.test_mask].eq(data.y[data.test_mask]).sum().item()
        return correct / (len(data.y[data.test_mask]))

In [53]:
# import torch.nn as nn
def train(data, plot=False):
#     train_accuracies, test_accuracies = list(), list()
    train_accuracies, test_accuracies = [], []

    for epoch in range(400):
            model.train()
            optimizer.zero_grad()
            out = model(data)
            print('output by softmax:',out)
            print('output with mask:',out[data.train_mask])
            print('output with mask for single instance:',out[data.train_mask][1]) 
#             temp=out[data.train_mask][1]
            temp=[]
            for i in range (120708):
                print('maximum pro of 1st', max(out[data.train_mask][i]))
                temp.append(max(out[data.train_mask][i]))
            
            print("sum of max pro:",sum(temp))
            print('train_y_data:',data.y[data.train_mask])
            print('lenghth of the outlist by softmax:',len(out[data.train_mask]))
            print('length of  the y:',len(data.y[data.train_mask]))
            print('outp:',out[data.train_mask]) 
            loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
            print('first loss',loss)
#             print


#             loss= torch.nn.KLDivLoss(out[data.train_mask], data.y[data.train_mask])
#             loss = nn.CrossEntropyLoss(out[data.train_mask], data.y[data.train_mask])
#             loss= F.torch.nn.HingeEmbeddingLoss(out[data.train_mask], data.y[data.train_mask])
#             loss= F.torch.nn.HingeEmbeddingLoss()
            # Mean Absolute Error (L1 Loss Function)
#             loss = F.L1Loss(out[data.train_mask], data.y[data.train_mask])
#             loss=  F.torch.nn.L1Loss(out[data.train_mask], data.y[data.train_mask])
#             loss= F.torch.nn.MSELoss(out[data.train_mask], data.y[data.train_mask])
#             loss = torch.nn.CrossEntropyLoss(out[data.train_mask], data.y[data.train_mask])
            loss.backward()
            optimizer.step()

            train_acc = test(data)
            test_acc = test(data, train=False)

            train_accuracies.append(train_acc)
            test_accuracies.append(test_acc)
            print('Epoch: {:03d}, Loss: {:.5f}, Train Acc: {:.5f}, Test Acc: {:.5f}'.
                  format(epoch, loss, train_acc, test_acc))

    # torch.save(model.state_dict(), 'new_model_weights.pth')

In [54]:
# if plot:
#  plt.plot(train_accuracies, label="Train accuracy")
#  plt.plot(test_accuracies, label="Validation accuracy")
#  plt.xlabel("# Epoch")
#  plt.ylabel("Accuracy")
#  plt.legend(loc='upper right')
#  plt.show()

In [55]:
# for i in dt:
#     print(i)

In [56]:
# len(dt.y.tolist())

In [57]:
# dt.x[0]

In [58]:
dt
# dt=dt.to(device)

Data(x=[120708, 11], edge_index=[2, 189113], y=[120708], train_mask=[120708], val_mask=[120708], test_mask=[120708])

In [59]:
model = Net(dt)
# data = dataset[0].to(device)

In [60]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cuda')

In [61]:
# model = model.to(device)

In [62]:
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [63]:
torch.cuda.is_available()

False

In [64]:
# import torch
# torch.cuda.empty_cache()

In [65]:
train(dt,False)

output by softmax: tensor([[ -3.4613, -12.6696,  -9.4835,  ...,  -6.6047,  -8.0537,  -9.0237],
        [ -8.0409, -23.1310, -18.5099,  ..., -12.6869, -11.3437, -10.7758],
        [-14.4997, -12.9770, -15.1314,  ..., -18.9697,  -9.4728, -13.1477],
        ...,
        [ -5.1932,  -5.6133,  -5.1732,  ...,  -5.5118,  -4.9336,  -5.0244],
        [ -5.4060,  -5.1698,  -5.1789,  ...,  -5.1312,  -5.0015,  -5.1844],
        [ -5.3437,  -5.3031,  -5.2396,  ...,  -5.0519,  -5.1042,  -5.1695]],
       grad_fn=<LogSoftmaxBackward0>)
output with mask: tensor([[ -3.4613, -12.6696,  -9.4835,  ...,  -6.6047,  -8.0537,  -9.0237],
        [ -8.0409, -23.1310, -18.5099,  ..., -12.6869, -11.3437, -10.7758],
        [-14.4997, -12.9770, -15.1314,  ..., -18.9697,  -9.4728, -13.1477],
        ...,
        [ -5.3398,  -5.2436,  -5.1972,  ...,  -5.1471,  -5.0291,  -5.1697],
        [-16.0082, -27.9287, -31.0106,  ..., -24.9650,  -7.1799, -11.7040],
        [ -7.2652, -10.0011,  -6.0285,  ...,  -4.3212, -14.254

maximum pro of 1st tensor(-1.7944, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-0.8660, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-0.2743, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-4.8834, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-0.3147, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-2.9607, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-0.0774, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-1.2885, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-0.3460, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-3.7001, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-4.7585, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-1.9745, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-3.6332, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-2.2929, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-3.4959, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-0.1505, grad_fn=<UnbindBackward0>)
maximum 

maximum pro of 1st tensor(-3.8736, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-0.6112, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-2.4073, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-9.0237e-05, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-0.0002, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-1.8164, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-3.0333, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-2.9278, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-0.8791, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-0.8126, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-3.3103, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-0.0732, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-3.4365, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-0.0006, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-5.0773, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-0.4934, grad_fn=<UnbindBackward0>)
maxi

maximum pro of 1st tensor(-4.2153, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-2.8380, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-4.7627, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-3.0074, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-4.8402, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-3.8836, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-4.7502, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-3.5877, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-0.0221, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-4.5452, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-3.8221, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-4.5633, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-4.0342, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-5.0121, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-1.7358, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-3.1098, grad_fn=<UnbindBackward0>)
maximum 

maximum pro of 1st tensor(-2.7067, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-1.3485, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-1.0162, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-0.7778, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-1.0202, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-5.0295, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-4.3855, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-1.5844, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-0.3488, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-2.6493, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-4.4190, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-3.5839, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-0.8902, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-3.9831, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-2.3635, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-1.0787, grad_fn=<UnbindBackward0>)
maximum 

maximum pro of 1st tensor(-2.5439, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-2.6817, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-4.3108, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-0.3469, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-4.2152, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-4.9328, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-2.1755, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-1.7157, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-0.7769, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-0.4472, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-3.5235, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-1.0916, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-3.3435, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-0.6697, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-0.4206, grad_fn=<UnbindBackward0>)
maximum pro of 1st tensor(-0.8874, grad_fn=<UnbindBackward0>)
maximum 

RuntimeError: [enforce fail at C:\cb\pytorch_1000000000000\work\c10\core\impl\alloc_cpu.cpp:81] data. DefaultCPUAllocator: not enough memory: you tried to allocate 55091392 bytes.

In [ ]:
# saving complete model
torch.save(model, 'KP_GNN_model_weights.pth')

In [ ]:
# load model
model = torch.load('KP_GNN_model_weights.pth')

In [ ]:
# from torch_geometric.data import Data
# print(dataset[0])
# import numpy as np
# x=np.ones(shape=(1,1433))
# xne=dataset.data.x[0:2708]
# # print(dataset.data.edge_index.shape)
# # xne=xne.unsqueeze(0)
# # print(dataset.data.edge_index.shape)
# e=dataset.data.edge_index
# # e=e.unsqueeze(0)
# print(xne.shape)
# print(e.shape)

# data = Data(x=xne, edge_index=e)
# print(data)

# y_pred=model(data)
# y_pred[0]


In [ ]:
import torch
import torch.nn as nn

# size of input (N x C) is = 3 x 5
input = torch.randn(3, 5, requires_grad=True)
# every element in target should have 0 <= value < C
target = torch.tensor([1, 0, 4])

m = nn.LogSoftmax(dim=1)
nll_loss = nn.NLLLoss()
output = nll_loss(m(input), target)
output.backward()

print('input: ', input)
print('target: ', target)
print('output: ', output)

In [ ]:
-np.log(1.4977)
max(input[0])

In [ ]:
import numpy as np
a=-np.log(1.4977)
b=-np.log(1.7851)
c=-np.log(0.5969)
print(-np.log(1.4977))
print(-np.log(1.7851))
print(-np.log(0.5969))
print(a+b+c)

In [ ]:
-np.log(0.71)

In [ ]:
np.log(np.abs(input[0][1].detach().numpy()))+np.log(np.abs(input[1][0].detach().numpy()))+np.log(np.abs(input[2][4].detach().numpy()))np.log(np.abs(input[0][1].detach().numpy()))+np.log(np.abs(input[1][0].detach().numpy()))+np.log(np.abs(input[2][4].detach().numpy()))

In [ ]:
# other model

In [ ]:
!pip install node2vec

In [3]:
import networkx as nx
from node2vec import Node2Vec

ModuleNotFoundError: No module named 'node2vec'

In [2]:
graph= nx.read_adjlist(r"C:\Users\tubaid\Desktop\usman.adjlist", nodetype=int)

NameError: name 'nx' is not defined

In [ ]:


# Create a graph
graph = nx.fast_gnp_random_graph(n=100, p=0.5)

# Precompute probabilities and generate walks - **ON WINDOWS ONLY WORKS WITH workers=1**
node2vec = Node2Vec(graph, dimensions=64, walk_length=30, num_walks=200, workers=4)  # Use temp_folder for big graphs

# Embed nodes
model = node2vec.fit(window=10, min_count=1, batch_words=4)  # Any keywords acceptable by gensim.Word2Vec can be passed, `dimensions` and `workers` are automatically passed (from the Node2Vec constructor)

# Look for most similar nodes
model.wv.most_similar('2')  # Output node names are always strings

# Save embeddings for later use
model.wv.save_word2vec_format(EMBEDDING_FILENAME)

# Save model for later use
model.save(EMBEDDING_MODEL_FILENAME)

# Embed edges using Hadamard method
from node2vec.edges import HadamardEmbedder

edges_embs = HadamardEmbedder(keyed_vectors=model.wv)

# Look for embeddings on the fly - here we pass normal tuples
edges_embs[('1', '2')]
''' OUTPUT
array([ 5.75068220e-03, -1.10937878e-02,  3.76693785e-01,  2.69105062e-02,
       ... ... ....
       ..................................................................],
      dtype=float32)
'''

# Get all edges in a separate KeyedVectors instance - use with caution could be huge for big networks
edges_kv = edges_embs.as_keyed_vectors()

# Look for most similar edges - this time tuples must be sorted and as str
edges_kv.most_similar(str(('1', '2')))

# Save embeddings for later use
edges_kv.save_word2vec_format(EDGES_EMBEDDING_FILENAME)